# Load SageMaker SSD trained model for local inference or on-prime


### Clone mxnet source code
Ref: https://stackoverflow.com/questions/52889337/mxnet-sagemaker-load-model

1. clone mxnet source code
2. cp model_algo_1-xxxxx both json and para to model folder, incubator-mxnet/example/ssd/model/
3. deploy sm model to mxnet format
4. load model and inference

In [4]:
%%bash
git clone https://github.com/apache/incubator-mxnet

Cloning into 'incubator-mxnet'...
bash: line 3: cd: incubator-mxnetexamplessd: No such file or directory


In [8]:
import boto3
sm = boto3.client('sagemaker')
response = sm.describe_training_job(TrainingJobName='object-detection-2020-04-13-09-45-08-700')
model_path = response['ModelArtifacts']['S3ModelArtifacts']
model_path

's3://beyoung-sagemaker/coco-objectDetection/output/object-detection-2020-04-13-09-45-08-700/output/model.tar.gz'

In [11]:
!aws s3 cp {model_path} ./model.tar.gz

download: s3://beyoung-sagemaker/coco-objectDetection/output/object-detection-2020-04-13-09-45-08-700/output/model.tar.gz to ./model.tar.gz


In [12]:
## cp model_algo_1-xxxxx both json and para to model folder, incubator-mxnet/example/ssd/model/
!tar -xvf ./model.tar.gz -C ./incubator-mxnet/example/ssd/model/

model_algo_1-0000.params
model_algo_1-symbol.json
hyperparams.json


In [13]:
!ls ./incubator-mxnet/example/ssd/model/

hyperparams.json  model_algo_1-0000.params  model_algo_1-symbol.json  README.md


In [ ]:
# python deploy.py --network resnet50 --prefix model/model_algo_1 --num-class YOUR_NUM_CLASS --data-shape 512

# Load Model as checkpoint file and predict

In [5]:
# change notebook working directory
%cd ./incubator-mxnet/example/ssd

/home/ec2-user/SageMaker/object_detection_pascalvoc_coco_2020-04-13/incubator-mxnet/example/ssd


In [6]:
#deploy model as checkpoint format
!python deploy.py --network resnet50 --prefix model/model_algo_1 --num-class 80 --data-shape 512

[15:01:18] src/nnvm/legacy_json_util.cc:209: Loading symbol saved by previous version v1.4.1. Attempting to upgrade...
[15:01:18] src/nnvm/legacy_json_util.cc:217: Symbol successfully upgraded!
Saved model: model/deploy_model_algo_1-0000.params
Saved symbol: model/deploy_model_algo_1-symbol.json


In [7]:
# load model and predicate
import mxnet as mx
import numpy as np
import cv2
from collections import namedtuple

In [8]:
# define test data
batch_size = 1
num_batch = 1
filepath = './data/demo/dog.jpg'
DEFAULT_INPUT_SHAPE = 512

In [9]:
# load model
sym, arg_params, aux_params = mx.model.load_checkpoint("model/deploy_model_algo_1", 0)
mod = mx.mod.Module(symbol=sym, context=mx.cpu(), data_names=["data"], label_names=[])
print('data_names:', mod.data_names)
print('output_names:', mod.output_names)

data_names: ['data']
output_names: ['detection_output']


In [10]:
mod.bind(data_shapes=[("data", (1, 3, DEFAULT_INPUT_SHAPE, DEFAULT_INPUT_SHAPE))], for_training=False,label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params)  # , allow_missing=True

In [11]:
img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
print(img.shape)
img = cv2.resize(img, (DEFAULT_INPUT_SHAPE, DEFAULT_INPUT_SHAPE))
img = np.swapaxes(img, 0, 2)
img = np.swapaxes(img, 1, 2)
img = img[np.newaxis, :]
print(img.shape)

(576, 768, 3)
(1, 3, 512, 512)


In [12]:
# forward
Batch = namedtuple('Batch', ['data'])
mod.forward(Batch([mx.nd.array(img)]))
prob = mod.get_outputs()[0].asnumpy()
prob = np.squeeze(prob)
# Grab top result, convert to python list of lists and return
results = [prob[i].tolist() for i in range(10)]
print(results)

[[0.0, 0.16453392803668976, 0.09659675508737564, 0.15072667598724365, 0.13017813861370087, 0.20379900932312012], [-1.0, 0.1562827229499817, 0.09097035229206085, 0.13830074667930603, 0.13425590097904205, 0.21097543835639954], [2.0, 0.14783094823360443, 0.5960919857025146, 0.118517205119133, 0.8743573427200317, 0.2753128409385681], [0.0, 0.1355995535850525, 0.09374825656414032, 0.1348811835050583, 0.1436205357313156, 0.19442342221736908], [-1.0, 0.1348533034324646, 0.08366208523511887, 0.1298636496067047, 0.14302274584770203, 0.2083708941936493], [0.0, 0.11463776230812073, 0.007977217435836792, 0.15076851844787598, 0.5285791158676147, 0.9667272567749023], [0.0, 0.11266519129276276, 0.08402879536151886, 0.14612749218940735, 0.10510846972465515, 0.21115216612815857], [0.0, 0.110567107796669, 0.06819647550582886, 0.03554457426071167, 0.9289578199386597, 0.9737381339073181], [-1.0, 0.10971880704164505, 0.0833270475268364, 0.15310338139533997, 0.10071104019880295, 0.1986367404460907], [0.0, 0

In [ ]:
# # predict
# eval_data = np.array([img])
# eval_label = np.zeros(len(eval_data)) # just need to be the same length, empty is ok
# eval_iter = mx.io.NDArrayIter(eval_data, eval_label, batch_size, shuffle=False)
# print('eval_iter.provide_data:', eval_iter.provide_data)
# print('eval_iter.provide_label:', eval_iter.provide_label)
# predict_stress = mod.predict(eval_iter, num_batch)
# print(predict_stress) # you can transfer to numpy array